In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import os
import numpy as np
import pyspark.sql.functions as f
import subprocess
from pyspark.sql.functions import min, max, col

In [2]:
sc = SparkContext()
sqlcontext = SQLContext(sc)

In [3]:
dir_in = "/user/hadoop/data/population"
args = "hdfs dfs -ls "+dir_in+" | awk '{print $8}'"
proc = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

s_output, s_err = proc.communicate()
all_dart_dirs = s_output.decode('utf-8').split()
print(all_dart_dirs)

['/user/hadoop/data/population/PEOPLE_DONG_201701.csv', '/user/hadoop/data/population/PEOPLE_DONG_201702.csv', '/user/hadoop/data/population/PEOPLE_DONG_201703.csv', '/user/hadoop/data/population/PEOPLE_DONG_201704.csv', '/user/hadoop/data/population/PEOPLE_DONG_201705.csv', '/user/hadoop/data/population/PEOPLE_DONG_201706.csv', '/user/hadoop/data/population/PEOPLE_DONG_201707.csv', '/user/hadoop/data/population/PEOPLE_DONG_201708.csv', '/user/hadoop/data/population/PEOPLE_DONG_201709.csv', '/user/hadoop/data/population/PEOPLE_DONG_201710.csv', '/user/hadoop/data/population/PEOPLE_DONG_201711.csv', '/user/hadoop/data/population/PEOPLE_DONG_201712.csv', '/user/hadoop/data/population/PEOPLE_DONG_201801.csv', '/user/hadoop/data/population/PEOPLE_DONG_201802.csv', '/user/hadoop/data/population/PEOPLE_DONG_201803.csv', '/user/hadoop/data/population/PEOPLE_DONG_201804.csv', '/user/hadoop/data/population/PEOPLE_DONG_201805.csv', '/user/hadoop/data/population/PEOPLE_DONG_201806.csv', '/user/ha

In [ ]:
df = sqlcontext.read.format("com.databricks.spark.csv")\
    .option("header", "true").option("inferSchema", "true").load(all_dart_dirs)

In [ ]:
df.show()
df.count()

In [ ]:
total_df = df.withColumn('total', f.expr(' Korean+Foreigner'))

In [ ]:
total_df.show()

In [ ]:
total_df = total_df.groupBy("Dong").agg(f.sum('total'))

In [ ]:
total_df.show()
print(total_df.count())

In [ ]:
_min, _max = total_df.select(min("sum(total)"), max("sum(total)")).first()
print(_min, _max)
save_df = total_df.withColumn("scale", (col("sum(total)") - _min) / (_max - _min))
save_df.show()

In [ ]:
# = save_df.select(["Dong", "scale"])
save_df.sort('Dong').write.csv("0611_t2")

In [ ]:
sc.textFile("hdfs://master:9000/user/hadoop/0611_t2/part*").coalesce(1).saveAsTextFile("hdfs://master:9000/user/hadoop/0611_t2_")